In [19]:
import numpy as np
import cv2 as cv
from scipy.optimize import linear_sum_assignment

cost = np.array([
    [0, 1, 2],
    [1, 0, 2],
    [1, 0, 2],
])
cost

array([[0, 1, 2],
       [1, 0, 2],
       [1, 0, 2]])

In [20]:
row_ind, col_ind = linear_sum_assignment(cost)
row_ind, col_ind

(array([0, 1, 2]), array([0, 1, 2], dtype=int64))

In [13]:
row_ind

array([0, 1, 2])

In [14]:
image = cv.imread('../data/cars.jpg')

def draw_labeled_rectangle(image, start_point, end_point, label, color):
    (x1, y1), (x2, y2) = start_point, end_point

    font_face = cv.FONT_HERSHEY_COMPLEX
    font_scale = 1
    font_thickness = 1

    text_size, _ = cv.getTextSize(label, font_face, font_scale, font_thickness)
    print(text_size)
    width, height = x2 - x1, y2 - y1
    roi = image[y1:y2, x1:x2]
    rect = np.ones_like(roi) * 255
    res = cv.addWeighted(roi, 0.5, rect, 0.5, 0)
    image[y1:y2, x1:x2] = res

draw_labeled_rectangle(image, (100, 100), (400, 400), 'something', None)

cv.imshow('Preview', image)
cv.waitKey(0)
cv.destroyAllWindows()

(179, 22)


TypeError: 'NoneType' object is not subscriptable